In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd

In [4]:
%cd "/content/drive/MyDrive/AI임베디드"

/content/drive/MyDrive/AI임베디드


In [5]:
import cv2
import os
import numpy as np

# Set the path to the directory containing the JPG images
directory_path_good = "canny_good/"
directory_path_bad="canny_bad/"
# Initialize an empty NumPy array to store the image data
image_data_good = np.zeros((200, 480, 640, 1), dtype=np.float32)
image_data_bad = np.zeros((200, 480, 640, 1), dtype=np.float32)

# Loop through each image in the directory
for i in range(100):
    # Construct the file path for the current image
    file_path_good1 = os.path.join(directory_path_good, f"car_{i}.jpg")#good tire
    file_path_bad1 = os.path.join(directory_path_bad, f"car_{i}.jpg")#severe worn out tire
    file_path_good2 = os.path.join(directory_path_good, f"canny_good_{i}.jpg")#good tire
    file_path_bad2 = os.path.join(directory_path_bad, f"canny_bad_{i}.jpg")#partial worn out tire

    img_good1 = cv2.imread(file_path_good1, cv2.IMREAD_GRAYSCALE)
    img_bad1 = cv2.imread(file_path_bad1, cv2.IMREAD_GRAYSCALE)
    img_good2 = cv2.imread(file_path_good2, cv2.IMREAD_GRAYSCALE)
    img_bad2 = cv2.imread(file_path_bad2, cv2.IMREAD_GRAYSCALE)

    image_data_good[i, :, :, 0] = img_good1.astype(np.float32) / 255.0
    image_data_bad[i, :, :, 0] = img_bad1.astype(np.float32) / 255.0
    image_data_good[i+100, :, :, 0] = img_good2.astype(np.float32) / 255.0
    image_data_bad[i+100, :, :, 0] = img_bad2.astype(np.float32) / 255.0











# Print the shape of the resulting image data
print("Shape of the image data:", image_data_good.shape)
print("Shape of the image data:", image_data_bad.shape)

Shape of the image data: (200, 480, 640, 1)
Shape of the image data: (200, 480, 640, 1)


In [6]:
images = np.vstack((image_data_good,image_data_bad))
images.shape

(400, 480, 640, 1)

In [7]:
image_label_good=np.hstack((np.ones((200,1)),np.zeros((200,1))))
image_label_bad = np.hstack((np.zeros((200,1)),np.ones((200,1))))

labels=np.vstack((image_label_good,image_label_bad))
print(labels.shape)

(400, 2)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42, shuffle=True)

In [9]:
# Define a model
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
class mnist_lenet(tf.keras.Model):
    def __init__(self):
        super(mnist_lenet, self).__init__()
        self.conv1 = layers.Conv2D(filters=10, kernel_size=[3,3], input_shape = (640,480,1), activation= 'relu')
        self.pool1 = layers.MaxPooling2D(2, 2)
        self.conv2 = layers.Conv2D(filters=20, kernel_size=[3,3], activation= 'relu')
        self.pool2 = layers.MaxPooling2D(2, 2)
        self.conv3 = layers.Conv2D(filters=30, kernel_size=[3,3], activation= 'relu')
        self.flat = layers.Flatten()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dense2 = layers.Dense(2, activation='softmax')


    def call(self, x):
        net = self.conv1(x)
        net = self.pool1(net)
        net = self.conv2(net)
        net = self.pool2(net)
        net = self.conv3(net)
        net = self.flat(net)
        net = self.dense1(net)
        net = self.dense2(net)
        return net

In [10]:
# Train & save model in frozen(.pb) format.

my_model = mnist_lenet()
my_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
my_model.fit(X_train, y_train, batch_size=1, epochs=5, verbose=1)
my_model.summary()


Epoch 1/5
320/320 [==============================] - 253s 783ms/step - loss: 0.3245 - accuracy: 0.9531
Epoch 2/5
320/320 [==============================] - 234s 732ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 3/5
320/320 [==============================] - 229s 716ms/step - loss: 2.7481e-05 - accuracy: 1.0000
Epoch 4/5
320/320 [==============================] - 225s 704ms/step - loss: 1.4795e-05 - accuracy: 1.0000
Epoch 5/5
320/320 [==============================] - 223s 696ms/step - loss: 7.5837e-06 - accuracy: 1.0000
Model: "mnist_lenet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  100       
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                        

In [12]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score
from keras.utils import to_categorical
# Predictions on the test set
y_pred = my_model.predict(X_test)

# Convert probabilities to binary predictions (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# Evaluate the model
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_binary.argmax(axis=1))
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred_binary.argmax(axis=1))
recall = recall_score(y_test.argmax(axis=1), y_pred_binary.argmax(axis=1), average='binary')
f1 = f1_score(y_test.argmax(axis=1), y_pred_binary.argmax(axis=1), average='binary')

print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1 Score:", f1)

3/3 [==============================] - 16s 4s/step
Confusion Matrix:
[[44  0]
 [ 0 36]]
Accuracy: 1.0
Recall: 1.0
F1 Score: 1.0


임의의 이미지 불러와서 추론하기

In [14]:
import cv2
import os
import numpy as np
import tensorflow as tf
my_model=my_model
directory=os.path.join("canny_good/car_50.jpg")
img_input = cv2.imread(directory, cv2.IMREAD_GRAYSCALE)

img_input = img_input.astype(np.float32)/255.0
img_input = np.reshape(img_input, (1, 480, 640, 1))
# img_input = pd.DataFrame(img_input)
# img_input.describe()
prediction = my_model.predict(img_input)

threshold = 0.5  # Adjust this threshold as needed
binary_prediction = (prediction > threshold).astype(int)

print("Raw Prediction:", prediction)
print("Binary Prediction:", binary_prediction)

if (binary_prediction[0][0]==1):
    print("Label: 0",)
else:
    print("Label: 1")

1/1 [==============================] - 0s 332ms/step
Raw Prediction: [[9.9999964e-01 3.3879024e-07]]
Binary Prediction: [[1 0]]
Label: 0


In [15]:
!pwd

/content/drive/MyDrive/AI임베디드


In [ ]:
# my_model.save("./tireD_lenet2_test")
my_model.save("./tireD_lenet2")

In [ ]:
# Load model and convert it for TensorFlow Lite (.tflite format)

import tensorflow as tf
import numpy as np

model_path = "./tireD_lenet2"
converter = tf.lite.TFLiteConverter.from_saved_model(model_path)
tflite_model = converter.convert()


In [ ]:
import pathlib

# Create folder to save model.
tflite_models_dir = pathlib.Path("./tmp/tireD_lenet2/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"tireD_lenet2.tflite"
tflite_model_file.write_bytes(tflite_model)

139011356